In [1]:
import boto3
from dotenv import load_dotenv
from datetime import datetime
import os
import pandas as pd

In [2]:
load_dotenv()

CSV_FILE_NAME_GENERATED_TODAY = f"rates-{datetime.today().date()}.csv".replace("-", "_")
AWS_ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
AWS_REGION_NAME = os.getenv('AWS_REGION_NAME')
AWS_S3_BUCKET_NAME = os.getenv('AWS_S3_BUCKET_NAME')

In [3]:
s3_client: boto3.client = boto3.client('s3',
                                           aws_access_key_id=AWS_ACCESS_KEY_ID,
                                           aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
                                           region_name=AWS_REGION_NAME)

In [4]:
s3_client.list_objects(Bucket=AWS_S3_BUCKET_NAME)['Contents']

[{'Key': 'data.csv',
  'LastModified': datetime.datetime(2023, 8, 14, 15, 52, 8, tzinfo=tzutc()),
  'ETag': '"cceb91e9497b1909d5ca4ed80d33f272"',
  'Size': 5829,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}},
 {'Key': 'rates_2023_08_13.csv',
  'LastModified': datetime.datetime(2023, 8, 14, 15, 52, 10, tzinfo=tzutc()),
  'ETag': '"7d21a50dd7260d530e7b8ba57672d81e"',
  'Size': 5822,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}},
 {'Key': 'rates_2023_08_14.csv',
  'LastModified': datetime.datetime(2023, 8, 14, 15, 52, 11, tzinfo=tzutc()),
  'ETag': '"870f4808df647dce08cf0d051ecc6ccf"',
  'Size': 5814,
  'StorageClass': 'STANDARD',
  'Owner': {'ID': 'da61778a50b21a24e4edce8c6997bb75c1652198d471d19f2cacb180ec255eb3'}}]

In [5]:
import io
def get_file(file_name: str = CSV_FILE_NAME_GENERATED_TODAY) -> io.StringIO:
    try:
        print(file_name)
        response = s3_client.get_object(Bucket=AWS_S3_BUCKET_NAME, Key=file_name)
        csv_content = response['Body'].read().decode('utf-8')
        return io.StringIO(csv_content)
    except Exception as e:
        print(e)
        return None


In [6]:
column_name: str = f"{datetime.today().date()}"
rates_data:io.StringIO = get_file("data.csv")
rate_today:io.StringIO = get_file()

data.csv
rates_2023_08_14.csv


In [7]:
rates_data_df:pd.DataFrame = None
rate_today_df:pd.DataFrame = pd.read_csv(rate_today,index_col=0, header=0)

In [8]:
if rates_data == None:
    rates_data_df = pd.DataFrame(rate_today_df.rename(columns={'rates': column_name}))

In [9]:
rates_data_df = pd.read_csv(rates_data, index_col=0, header=0)
rates_data_df

,symbols,countries,2023-08-13
id,,,
0,AED,United Arab Emirates Dirham,4.027006
1,AFN,Afghan Afghani,92.483124
2,ALL,Albanian Lek,103.792284
3,AMD,Armenian Dram,426.368302
4,ANG,Netherlands Antillean Guilder,1.981153
...,...,...,...
165,YER,Yemeni Rial,274.475398
166,ZAR,South African Rand,20.754214
167,ZMK,Zambian Kwacha (pre-2013),9868.627272


In [10]:
rate_today_df

,symbols,countries,rates
id,,,
0,AED,United Arab Emirates Dirham,3.996698
1,AFN,Afghan Afghani,91.594688
2,ALL,Albanian Lek,103.724031
3,AMD,Armenian Dram,420.673733
4,ANG,Netherlands Antillean Guilder,1.962113
...,...,...,...
165,YER,Yemeni Rial,272.407953
166,ZAR,South African Rand,20.823884
167,ZMK,Zambian Kwacha (pre-2013),9794.303172


In [11]:
rates_data_df[column_name] = rate_today_df['rates']

In [12]:
rates_data_df

,symbols,countries,2023-08-13,2023-08-14
id,,,,
0,AED,United Arab Emirates Dirham,4.027006,3.996698
1,AFN,Afghan Afghani,92.483124,91.594688
2,ALL,Albanian Lek,103.792284,103.724031
3,AMD,Armenian Dram,426.368302,420.673733
4,ANG,Netherlands Antillean Guilder,1.981153,1.962113
...,...,...,...,...
165,YER,Yemeni Rial,274.475398,272.407953
166,ZAR,South African Rand,20.754214,20.823884
167,ZMK,Zambian Kwacha (pre-2013),9868.627272,9794.303172
